<a href="https://colab.research.google.com/github/Shuaib11-Github/AI_Agents/blob/main/Recipe_Generation_from_Images_Using_AI_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Recipe Generation from Images Using AI Tools**

## **Introduction**
Imagine seeing a captivating food image and instantly learning how to prepare it. This tutorial will guide you in building an AI-powered application capable of identifying dishes from images and generating detailed recipes using advanced AI tools like OpenAI's GPT-4, LangChain, Gradio, and ChromaDB.

By the end of this tutorial, you'll have a robust system that takes a food image as input and provides a recipe with optional features like cooking tips, dietary variations, and pairing suggestions.

---

## **Prerequisites**
Before we start, ensure you have the required libraries installed. If you are using Google Colab, you can run the following commands:


In [ ]:
# Install required libraries
!pip install openai==0.28 gradio langchain chromadb python-dotenv Pillow requests langchain_community tiktoken httpx==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.5 MB/s eta 0:00:00

## Step 1: Import Libraries and Set Up API Keys
We'll begin by importing necessary libraries and setting up the OpenAI API key.

In [ ]:
import os
import openai
import gradio as gr
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from PIL import Image
import base64
from datetime import datetime
import hashlib
import uuid
import chromadb
# Set up OpenAI API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('openai')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

## Step 2: Recipe Generator Class
We’ll create a `RecipeGenerator` class to analyze images, generate recipes, and store them.

In [ ]:
class RecipeGenerator:
    def __init__(self):
        self.llm = OpenAI(temperature=0.7)
        self.embeddings = OpenAIEmbeddings()
        self.db = self._initialize_database()

    def _initialize_database(self):
        """Initialize ChromaDB for storing recipes."""
        client = chromadb.Client()
        collection = client.get_or_create_collection(name="recipes")
        return collection

    def analyze_image(self, image_path):
        """Analyze food image using GPT-4 Vision."""
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode()

        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a professional chef and recipe expert."},
                {"role": "user", "content": [
                    {"type": "text", "text": "Identify the dish in this image and list its main ingredients."},
                    {"type": "image_url", "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_string}"
                    }}
                ]}
            ],
            max_tokens=300
        )
        return response.choices[0].message["content"]

    def generate_recipe(self, dish_info):
        """Generate a detailed recipe based on dish identification."""
        template = """
        Create a detailed recipe for the following dish:
        {dish_info}

        Include:
        1. Recipe Name
        2. Preparation Time
        3. Cooking Time
        4. Servings
        5. Ingredients (with measurements)
        6. Step-by-step Instructions
        7. Cooking Tips
        8. Nutritional Information
        """
        prompt = PromptTemplate(input_variables=["dish_info"], template=template)
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(dish_info=dish_info)

## Step 3: Advanced Features
Extend `RecipeGenerator` to include dietary variations, cooking tips, and more.

In [ ]:
class AdvancedRecipeGenerator(RecipeGenerator):
    def __init__(self):
        super().__init__()
        self.setup_templates()

    def setup_templates(self):
        """Setup templates for additional features."""
        self.templates = {
            'variations': "Provide variations of {dish} for different dietary requirements (vegetarian, vegan, gluten-free).",
            'tips': "Share professional cooking tips and tricks for making {dish}.",
            'pairing': "Suggest beverage and side dish pairings for {dish}.",
            'scaling': "Provide instructions for scaling {dish} for different serving sizes."
        }

    def get_recipe_variations(self, dish):
        """Generate recipe variations."""
        prompt = PromptTemplate(input_variables=["dish"], template=self.templates['variations'])
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(dish=dish)

    def get_cooking_tips(self, dish):
        """Get cooking tips."""
        prompt = PromptTemplate(input_variables=["dish"], template=self.templates['tips'])
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(dish=dish)

## Step 4: Gradio Interface
Create an interactive user interface using Gradio.

In [ ]:
def create_recipe_interface():
    """Create a Gradio interface for recipe generation."""
    recipe_gen = AdvancedRecipeGenerator()

    def process_recipe_request(image, include_variations=False, include_tips=False):
        temp_path = "temp_image.jpg"
        image.save(temp_path)

        dish_info = recipe_gen.analyze_image(temp_path)
        result = f"Dish Information:\n{dish_info}\n\n"
        result += f"Recipe:\n{recipe_gen.generate_recipe(dish_info)}\n\n"

        if include_variations:
            result += f"Recipe Variations:\n{recipe_gen.get_recipe_variations(dish_info)}\n\n"
        if include_tips:
            result += f"Cooking Tips:\n{recipe_gen.get_cooking_tips(dish_info)}\n\n"

        return result

    interface = gr.Interface(
        fn=process_recipe_request,
        inputs=[
            gr.Image(type="pil", label="Upload Food Image"),
            gr.Checkbox(label="Include Recipe Variations"),
            gr.Checkbox(label="Include Cooking Tips"),
        ],
        outputs=gr.Textbox(label="Recipe Results", lines=20),
        title="AI Recipe Generator",
        description="Upload a food image to get detailed recipes, tips, and variations.",
        theme=gr.themes.Soft()
    )
    return interface

## Step 5: Main Application
Launch the application and share it.

In [ ]:
if __name__ == "__main__":
    interface = create_recipe_interface()
    interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd0124d246e12c955c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary
This tutorial demonstrated how to create a Recipe Generation application using AI tools. By uploading a food image, the app can identify the dish, generate recipes, and even provide cooking tips and dietary variations.

Feel free to expand this application by adding more features, such as saving recipes or integrating a database for recipe management.